In [6]:
# import libraries 
import pandas as pd
import numpy as np
import re

# Fix date values
from datetime import datetime, timedelta

In [7]:
# Read data
df = pd.read_csv('job_data.csv')

In [8]:
# Print first rows so we have an idea about our data
df.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,NaN,Full-time,PostedJust posted,Train volunteers on the use of M&E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,NaN,NaN,EmployerActive 6 days ago,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,NaN,NaN,EmployerActive 5 days ago,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,"₦150,000 - ₦200,000 a month",Full-time,PostedPosted 2 days ago,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,NaN,NaN,PostedPosted 1 day ago,Analyse business information/ data to identify...


In [9]:
# Getting more info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        203 non-null    object
 1   Company      200 non-null    object
 2   Location     203 non-null    object
 3   Salary       15 non-null     object
 4   Job Type     101 non-null    object
 5   Date Posted  203 non-null    object
 6   Summary      203 non-null    object
dtypes: object(7)
memory usage: 11.2+ KB


The Dtypes do not make sense, we should change the format of some of them.

In [10]:
# Check for null values
df.isnull().sum()

Title            0
Company          3
Location         0
Salary         188
Job Type       102
Date Posted      0
Summary          0
dtype: int64

We can either drop the rows with missing values or fill them. Let's start with the easy ones.

In [11]:
df['Company'].fillna('Unknown', inplace=True)
df['Job Type'].fillna('Not Specified', inplace=True)

Now, for the "Salary" column, we can either replace them with a placeholder "Not Disclosed", with the median salary of the dataset or drop them.
There's a big reason not to drop them:
- There are 288/203 missing values in this column

I'd say there's a good reason to put a placeholder here and call it a day. But for the sake of the exercise and practice, which is mostly what this project is for me, I'm going to convert everything to float and apply the median.

In [12]:
# Check values so we have a more accurate idea about our data
df['Salary'].unique()

array([nan, '₦150,000 - ₦200,000 a month', '₦100,000 - ₦150,000 a month',
       '₦200,000 a month', '₦70,000 - ₦150,000 a month',
       '₦90,000 - ₦130,000 a month', '₦300,000 - ₦350,000 a month',
       '₦450,000 - ₦600,000 a month', '₦200,000 - ₦250,000 a month',
       '₦220,000 a month', '₦60,000 - ₦80,000 a month'], dtype=object)

In [13]:
# Convert salary from str to float
def convert_salary(salary_col):
    if pd.isna(salary_col):
        return np.nan
    
    # Extract numeric data
    numbers = re.findall(r'\d+', salary_col.replace(',',''))
    if len(numbers) == 1:
        return float(numbers[0])
    elif len(numbers) == 2:
        return (float(numbers[0])+ float(numbers[1])) / 2
    else:
        return np.nan
    
# Apply function over our data
df['Salary'] = df['Salary'].apply(convert_salary)

# Fill nulls with mean
mean_salary = df['Salary'].mean()
df['Salary'].fillna(mean_salary, inplace=True)

In [14]:
# Check data after all the changes
df.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,190000.0,Full-time,PostedJust posted,Train volunteers on the use of M&E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,190000.0,Not Specified,EmployerActive 6 days ago,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,190000.0,Not Specified,EmployerActive 5 days ago,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,175000.0,Full-time,PostedPosted 2 days ago,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,190000.0,Not Specified,PostedPosted 1 day ago,Analyse business information/ data to identify...


Now we can start focusing on cleaning the "Date Posted" column.

In [15]:
# Check values so we have a more accurate idea about our data
df['Date Posted'].unique()

array(['PostedJust posted', 'EmployerActive 6 days ago',
       'EmployerActive 5 days ago', 'PostedPosted 2 days ago',
       'PostedPosted 1 day ago', 'PostedPosted 6 days ago',
       'PostedPosted 30+ days ago', 'PostedPosted 26 days ago',
       'PostedPosted 12 days ago', 'EmployerActive 24 days ago',
       'EmployerActive 7 days ago', 'PostedPosted 7 days ago',
       'EmployerActive 19 days ago', 'PostedPosted 18 days ago',
       'PostedPosted 25 days ago', 'PostedPosted 13 days ago',
       'PostedPosted 19 days ago', 'PostedPosted 20 days ago',
       'PostedPosted 8 days ago', 'PostedPosted 14 days ago',
       'PostedPosted 11 days ago', 'PostedPosted 4 days ago',
       'PostedToday', 'PostedPosted 29 days ago',
       'EmployerActive 20 days ago', 'PostedPosted 28 days ago',
       'PostedPosted 10 days ago', 'PostedPosted 21 days ago',
       'PostedPosted 27 days ago', 'EmployerActive 15 days ago',
       'PostedPosted 15 days ago', 'PostedPosted 3 days ago',
       '

We can see there's 3 types: "Just Posted", "Today" and "X days ago". We'll need to format it to datetime

In [16]:
# Convert date from str to datetime
def convert_date(date_col):
    if pd.isna(date_col):
        return None
    
    date_col = date_col.lower()
    
    # Convert to datetime
    if "just posted" in date_col:
        return datetime.now()
    elif "today" in date_col:
        return datetime.now()
    elif "days ago" in date_col:
        days_ago = int(re.findall(r'\d+', date_col)[0])
        return datetime.now() - timedelta(days=days_ago)
    else:
        return pd.to_datetime(date_col, errors='coerce')
    
# Apply function over our data    
df['Date Posted'] = df['Date Posted'].apply(convert_date)    

In [17]:
# Check data after changes
df.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,190000.0,Full-time,2024-06-19 16:52:19.808142,Train volunteers on the use of M&E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,190000.0,Not Specified,2024-06-13 16:52:19.808142,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,190000.0,Not Specified,2024-06-14 16:52:19.808142,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,175000.0,Full-time,2024-06-17 16:52:19.808142,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,190000.0,Not Specified,NaT,Analyse business information/ data to identify...


Looks good. Time to look for duplicates and remove them.

In [18]:
# Check for duplicates
print(f"{df.duplicated().sum() = }")

# Remove duplicates
clean_df = df.drop_duplicates()

print(f"{clean_df.duplicated().sum() = }")


df.duplicated().sum() = 3
clean_df.duplicated().sum() = 0


We need to standarize the "Summary" column. Removing special characters and non-letters.

In [21]:
# Removes special characters and non-letters
def clean_summary(summary_col):
    return re.sub(r'[^a-zA-Z\s]', '', summary_col).lower()

# Apply function to our data
df['Summary'] = df['Summary'].apply(clean_summary)


In [22]:
# Check data after changes
df.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,190000.0,Full-time,2024-06-19 16:52:19.808142,train volunteers on the use of me frameworks a...
1,QA Tester - Intern,mDoc Healthcare,Lagos,190000.0,Not Specified,2024-06-13 16:52:19.808142,advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,190000.0,Not Specified,2024-06-14 16:52:19.808142,implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,175000.0,Full-time,2024-06-17 16:52:19.808142,analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,190000.0,Not Specified,NaT,analyse business information data to identify ...


Finally, we can save our data to a new ".csv" for future analysis.

In [23]:
# Save cleaned data to a new csv
df.to_csv("cleaned_job_data.csv", index=False)
